# Workshop_09

Il seguente workshop ha lo scopo di modellare un tetto partendo da un file .lines passato come input. Scopo del workshop è quello di creare le falde del tetto tutte con la stessa inclinazione, avendo come input esclusivamente la definizione del contorno esterno del tetto stesso.

## Creazione dei file .lines
Per prima cosa, con il supporto del programma [Inkscape](https://inkscape.org/it/), sono stati creati i file .svg che rappresentano il contorno del tetto. Un esempio di file utilizzato è nella figura successiva.
<img src="jupyter_images/workshop_09_linee_tetto.png">
Successivamente, utilizzando il tool [svg2lines](https://github.com/cvdlab/svg2lines), i file .svg sono stati trasformati in file .lines, i quali saranno l'input del codice sviluppato. Per poter essere correttamente utilizzati questi vanno posizionati nella cartella **"lines"** del progetto. Di seguito un esempio di file .lines creato partendo dal file .svg appena mostrato.  
  
### lines/workshop_09_linee_tetto_1.lines
35.692573,45.607694,35.692573,226.19885  
35.692573,226.19885,122.84190000000001,225.88423  
122.84190000000001,225.88423,122.84190000000001,174.91598  
122.84190000000001,174.91598,201.49656000000002,174.91598  
201.49656000000002,174.91598,201.49656000000002,44.663833  
35.692573,45.607694,201.49656000000002,44.663833  

## Risultati ottenuto
Partendo dal file .lines appena mostrato si ottiene il seguente risultato.  
  
<img src="jupyter_images/workshop_09_tetto_1.png">

## Altre esecuzioni
Si è anche scelto di modellare il tetto mansardato mostrato in figura.
  
<img src="jupyter_images/workshop_09_mansard_roof.jpg">
  
Il file .lines è ricavato sempre con il procedimento mostrato in precedenza, ottenendo il risultato mostrato in seguito.

### lines/workshop_09_linee_tetto_2.lines
0,0,25,0  
25,0,25,50  
25,50,0,50  
0,50,0,0     
  
In figura è infine mostrato il risultato ottenuto tramite l'esecuzione del programma sviluppato.

<img src="jupyter_images/workshop_09_tetto_2.png">

## Descrizione dei passaggi svolti nel codice
Il codice sviluppato per prima cosa legge il file .lines contenente le specifiche del tetto da realizzare creando così il poligono alla base. Successivamente ridimensiona tale poligono e creando così quella che sarà la mansarda del tetto. Viene quindi calcolata la direzione delle falde del tetto, partendo dal calcolo del centroide della figura. Il centroide si troverà sempre verso l'interno del poligono e le falde del tetto vengono orientate proprio verso il centroide. Infine, creati tutti i compomenti del tetto, questi vengono correttamente assemblati insieme e il risultato viene mostrato a video.

## Bug noti
Il codice sviluppato riesce a creare correttamente tetti che hanno una forma alla base concava o comunque con pochi angoli convessi.  
Infatti, nel caso di una figura particolare che presenta un numeto eccessivo di angoli convessi le falde create non presentano tutte la stessa inclinazione, contraddicendo le specifiche assegnate per lo svolgimento del workshop.

## Programma finale
Viene infine riportato nella sua iterezza il programma realizzato.

In [7]:
"""workshop_09
build a roof starting from a .lines file rappresenting the vertices of the roof's base

"""

import csv
import math
from itertools import *
from larlib import *

def makeUnique(original_list):

    uniqueList = []
    [uniqueList.append(obj) for obj in original_list if obj not in uniqueList]
    return uniqueList

def polygonArea(x, y):
    '''Calculates the area of polygon given its vertices'''
    area = 0.0
    for i in xrange(-1, len(x) - 1):
        area += x[i] * (y[i + 1] - y[i - 1])
    return area / 2.0

def polygonCentroid(points):
	'''Calculates the centroid of polygon given its vertices'''
	area = polygonArea(*zip(*points))
	resultX = 0
	resultY = 0
	N = len(points)
	points = cycle(points)
	x1, y1 = next(points)
	for i in range(N):
		x0, y0 = x1, y1
		x1, y1 = next(points)
		cross = (x0 * y1) - (x1 * y0)
		resultX += (x0 + x1) * cross
		resultY += (y0 + y1) * cross
	resultX /= (area * 6.0)
	resultY /= (area * 6.0)
	return (resultX, resultY)

def orderClockwise(points):

	firstPoint = points[0]
	centroid = [firstPoint[0],firstPoint[1]+0.0000001]
	print centroid
	firstLength = distance(centroid,firstPoint)
	newPoints = []

	while len(points)>0:
		cos = 400
		for j in range(len(points)):
			cos2 = findAngle(points[j],centroid)
			
			if cos2 < cos:
				cos = cos2
				p = points[j]
		newPoints.append(p)
		for t in range(len(points)):
			if points[t] == p:
				el = t
		
		points.pop(el)
		
	return newPoints 

def distance(p1,p2):
	'''return the distance between two given points'''
	return float(math.sqrt(math.pow((p1[0] - p2[0]), 2)+math.pow((p1[1] - p2[1]), 2)))

def findAngle(p1,p2):

	return math.atan2(p1[1]-p2[1],p1[0]-p2[0])

def ggpl_roof_builder(fileName):
	'''build a roof starting from a .lines file rappresenting the vertices of the roof's base
	@param: the name of the .lines file. It must be located in the lines folder
	@rerturn: the HPC value of the roof'''
	j=0
	s=0
	t=0
	firstShape = []
	falde = []
	topShape = []
	pol= []
	pol2 = []
	with open("lines/"+fileName+".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		polylineList = []
		reader2 = reader
		row1=next(reader2)
		px = row1[0]
		py = row1[1]
	with open("lines/"+fileName+".lines", "rb") as file:
		reader = csv.reader(file, delimiter=",")
		for row in reader:
			firstShape.append([float(row[0])-float(px),float(row[1])-float(py)])
			firstShape.append([float(row[2])-float(px),float(row[3])-float(py)])
	centroid = polygonCentroid(firstShape)
	[x,y] = centroid
	centroid = [x,y]

	for f in range(len(firstShape)):
		firstShape[f][0]=firstShape[f][0]-centroid[0]
		firstShape[f][1]=firstShape[f][1]-centroid[1]
	while j<len(firstShape):
		pol.append(POLYLINE([firstShape[j],firstShape[j+1]]))
		j=j+2
	pol = STRUCT(pol)

	for i in range(len(firstShape)):
		topShape.append([firstShape[i][0]/2.0,firstShape[i][1]/2.0])

	firstShape = makeUnique(firstShape)

	topShape = makeUnique(topShape)
	topShape= orderClockwise(topShape)
	firstShape = orderClockwise(firstShape)
	topShape = topShape + [topShape[0]]
	while t<len(topShape)-1:
		pol2.append(POLYLINE([topShape[t],topShape[t+1]]))
		t=t+1

	piano = SOLIDIFY(STRUCT(pol2))
	piano = T(3)(20)(piano)
	firstShape = firstShape+ [firstShape[0]]
	for p in range(len(firstShape)):
		firstShape[p]=firstShape[p]+[float(0)]
	for k in range(len(topShape)):
		topShape[k]=topShape[k]+[float(20)]

	while s<len(makeUnique(firstShape)):
		falde.append(MKPOL([[firstShape[s],firstShape[s+1],topShape[s],topShape[s+1]],[[1,2,3,4]],None]))
		s = s+1
	falde = STRUCT(falde)
	roof = STRUCT([falde,piano])
	roof = T([1,2])([centroid[0],centroid[1]])(roof)
	return roof

def main():
	VIEW(ggpl_roof_builder("workshop_09_linee_tetto_1"))
	VIEW(ggpl_roof_builder("workshop_09_linee_tetto_2"))


if __name__ == '__main__':
	main()
